In [1]:
import multiprocessing as mp
import timeit

In [3]:
from tqdm import tnrange, tqdm_notebook, tqdm
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import brewer

In [6]:
output_notebook()

Loading BokehJS ...

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
from numpyGillespie import CompleteGillespie as cg
from numpyGillespie import pandas_output

In [9]:
%lsmagic

Available line magics:
%aimport  %alias  %alias_magic  %autocall  %automagic  %autoreload  %autosave  %bookmark  %cd  %clear  %cls  %colors  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

In [11]:
def linear_complete_gsa(runs):
    X = [1000,]
    C = [0.5,]
    change = [lambda u: [u[0] - 1.0,],]
    perm = [lambda u: u[0],]
    
    exp_runs = [cg(X, C, change, perm, max_sim_rxn=1000).simulate() for i in range(runs)]
    return exp_runs

In [12]:
%timeit -n1 -r4 linear_complete_gsa(100)

1.82 s ± 6.76 ms per loop (mean ± std. dev. of 4 runs, 1 loop each)


In [67]:
def e_change_1(species_list):
    species_list[0] = species_list[0] - 1
    return

def p_calc_1(species_list):
    return species_list[0]

In [61]:
def GillespieProcess(species, rates, species_changes, permutations, max_sim_rxn=10000):
    return cg(species, rates, species_changes, permutations, max_sim_rxn).simulate()

In [70]:
def pool_build_complete_gsa(processes, runs):
    max_rxn = 1000
    pool = mp.Pool(processes=processes)
    results = [pool.apply_async(GillespieProcess, args=(X, C, [e_change_1], [p_calc_1], max_rxn)) for i in range(runs)]
    results = [p.get() for p in results]
    return results

In [71]:
pool_build_complete_gsa(3, 100)

TypeError: 'NoneType' object is not subscriptable